# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_deterministic.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data="collected_data/rl_deterministic_coarse.txt")
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset = get_dataset([i for i in range(2000)])

start
[ 0.00000000e+00  7.95731469e+08  1.89410892e-01 -5.99995308e-04
 -9.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.49650472e-01  5.60452370e-01 -6.00000000e-01]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.86510892e-01 -2.39999531e-03
 -2.92000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.93110892e-01  4.74000047e-02
 -2.36000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 3 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -9.10891077e-02 -2.85999953e-02
 -2.90000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -5.36786897e-02  6.00000000e-01]
Read chunk # 4 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.55589108e-01  1.14000047e-02
  1.12999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.67705737

start
[ 0.00000000e+00  7.95731469e+08  3.19108923e-02  3.62000047e-02
  9.89998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.30476298e-01  2.11530543e-02 -1.79693603e-01]
Read chunk # 43 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.51089229e-03  2.40000047e-02
  1.50999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.22928456e-01 -9.01716789e-02  5.24122209e-02]
Read chunk # 44 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.02710892e-01 -2.45999953e-02
  1.32999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.66016346e-01  1.86622984e-01 -6.00000000e-01]
Read chunk # 45 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.58789108e-01  4.60000047e-02
 -2.53000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -5.21367193e-01  6.00000000e-01]
Read chunk # 46 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.46989108e-01 -4.83999953e-02
  8.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.2479

[ 0.00000000e+00  7.95731469e+08 -8.08891077e-02 -4.51999953e-02
 -2.94000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -2.41049546e-02  5.99364821e-01]
Read chunk # 92 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.62289108e-01 -2.45999953e-02
 -1.09000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.04223805e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 93 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.58989108e-01  4.96000047e-02
 -2.58000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 94 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.01891077e-02 -9.39999531e-03
 -2.59000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  8.75229115e-02  2.46667824e-01]
Read chunk # 95 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.74889108e-01  3.68000047e-02
  2.28999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.88140679e-

[ 0.00000000e+00  7.95731469e+08  3.99710892e-01  5.98000047e-02
  1.71999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.38964924e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 141 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.75891077e-02  1.44000047e-02
  1.15999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.64104594e-01 -2.00211558e-01  4.83997914e-01]
Read chunk # 142 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.74110892e-01 -5.43999953e-02
 -5.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.47173217e-02  4.94179190e-01 -6.00000000e-01]
Read chunk # 143 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.69789108e-01  1.28000047e-02
 -2.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.11746518e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 144 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -9.79891077e-02  5.10000047e-02
  1.25999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.464996

[ 0.00000000e+00  7.95731469e+08  7.51108923e-02 -5.93999953e-02
 -2.11000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.48566820e-01  3.35279288e-01 -6.00000000e-01]
Read chunk # 186 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.06891077e-02 -4.79999531e-03
  1.42999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.15106846e-01 -1.19951500e-01  8.14678575e-02]
Read chunk # 187 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.63010892e-01 -1.43999953e-02
 -1.98000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.91210761e-01  5.59541096e-01 -6.00000000e-01]
Read chunk # 188 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.67789108e-01  3.30000047e-02
 -2.77000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -5.29719302e-01  6.00000000e-01]
Read chunk # 189 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  9.58108923e-02  4.18000047e-02
 -6.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.825622

...

[ 0.00000000e+00  7.95731469e+08 -3.90189108e-01  5.70000047e-02
  2.93999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.87970498e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1799 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.53810892e-01 -2.33999953e-02
  2.48999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1800 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.76189108e-01  1.40000047e-02
  1.89998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.95739654e-03 -6.00000000e-01  6.00000000e-01]
Read chunk # 1801 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.52489108e-01  5.80000469e-03
 -1.61000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.68929388e-01 -5.63986181e-01  6.00000000e-01]
Read chunk # 1802 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.17110892e-01 -4.99999531e-03
  1.16999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.43

[ 0.00000000e+00  7.95731469e+08  2.36510892e-01  1.20000469e-03
 -5.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.86118990e-02  6.00000000e-01 -6.00000000e-01]
Read chunk # 1844 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.05189108e-01  3.20000469e-03
  1.57999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.24890910e-01 -3.79726821e-01  6.00000000e-01]
Read chunk # 1845 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.48589108e-01 -1.65999953e-02
 -2.17000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.35098991e-01 -5.17572960e-01  6.00000000e-01]
Read chunk # 1846 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.42889108e-01  1.38000047e-02
  2.20999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.57293252e-01 -6.00000000e-01  6.00000000e-01]
Read chunk # 1847 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.31689108e-01 -2.99999531e-03
  1.56999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.46

[ 0.00000000e+00  7.95731469e+08  3.47810892e-01  3.16000047e-02
  1.77999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.67535557e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1889 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.00510892e-01  1.50000047e-02
 -1.13000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.72512902e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1890 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.17289108e-01  2.20000047e-02
  4.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.62137793e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 1891 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.06210892e-01  4.76000047e-02
  9.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.30619591e-02  4.87619279e-01 -6.00000000e-01]
Read chunk # 1892 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.70710892e-01 -3.69999953e-02
  1.61999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.95

start
[ 0.00000000e+00  7.95731469e+08 -1.63389108e-01 -4.49999953e-02
  2.01999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -5.62119046e-01  6.00000000e-01]
Read chunk # 1934 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.94891077e-02 -1.45999953e-02
  1.69999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.28791396e-01 -2.13576057e-01  3.15768357e-01]
Read chunk # 1935 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.26510892e-01 -4.79999531e-03
  2.78999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1936 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.48010892e-01 -3.69999953e-02
  2.25999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  2.46578685e-01 -6.00000000e-01]
Read chunk # 1937 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.22889108e-01 -3.85999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00


[ 0.00000000e+00  7.95731469e+08 -6.58891077e-02  8.60000469e-03
 -2.91000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  1.36612664e-02  6.00000000e-01]
Read chunk # 1980 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.95889108e-01  5.88000047e-02
  1.06999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.29117118e-02 -6.00000000e-01  6.00000000e-01]
Read chunk # 1981 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.15310892e-01  5.28000047e-02
 -8.30001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.39490084e-01  6.00000000e-01 -6.00000000e-01]
Read chunk # 1982 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.62410892e-01  1.40000047e-02
 -2.58000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  5.96622362e-01 -6.00000000e-01]
Read chunk # 1983 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.62910892e-01 -4.85999953e-02
  1.86999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00

In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -95.8347,
 'std': 62.380466,
 'min': -288.50906,
 'max': 0.0,
 'histogram': (array([  8,  12,  20,  23,  31,  39,  52,  79, 106,  92, 107, 119, 132,
         142, 139, 185, 204, 238, 265,   6]),
  array([-288.50906 , -274.08362 , -259.65814 , -245.2327  , -230.80725 ,
         -216.3818  , -201.95634 , -187.53088 , -173.10544 , -158.68    ,
         -144.25453 , -129.82907 , -115.403625, -100.97817 ,  -86.55272 ,
          -72.127266,  -57.701813,  -43.27636 ,  -28.850906,  -14.425453,
            0.      ], dtype=float32))}

In [5]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [6]:
from d3rlpy.algos import CQL

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [7]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.037819472543626755


In [8]:
%load_ext tensorboard
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [9]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=40, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-22 13:21.22 [debug    ] RoundIterator is selected.
2022-04-22 13:21.22 [info     ] Directory is created at d3rlpy_logs/CQL_20220422132122
2022-04-22 13:21.22 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-22 13:21.22 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-22 13:21.22 [warning  ] Skip building models since they're already built.
2022-04-22 13:21.22 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220422132122/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-22 13:33.41 [info     ] CQL_20220422132122: epoch=1 step=6928 epoch=1 metrics={'time_sample_batch': 0.0003420805201519444, 'time_algorithm_update': 0.10443223916493168, 'temp_loss': 1.2439730751121016, 'temp': 0.8144192236624056, 'alpha_loss': 8.933157939230755, 'alpha': 0.8295113597125434, 'critic_loss': 80.33013826481327, 'actor_loss': 50.652369707479465, 'time_step': 0.10508638499928402, 'td_error': 168.37741325843473, 'init_value': -131.1938018798828, 'ave_value': -119.46909325804828} step=6928
2022-04-22 13:33.41 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220422132122/model_6928.pt


Epoch 2/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-22 13:55.13 [info     ] CQL_20220422132122: epoch=2 step=13856 epoch=2 metrics={'time_sample_batch': 0.0003476064764049257, 'time_algorithm_update': 0.18429884376481828, 'temp_loss': -0.018478584296722556, 'temp': 0.7727976827247727, 'alpha_loss': 5.844538008385525, 'alpha': 0.44349783624137384, 'critic_loss': 458.05218048541707, 'actor_loss': 169.2142700884689, 'time_step': 0.18498408794403076, 'td_error': 301.03213845142466, 'init_value': -218.14340209960938, 'ave_value': -199.85732195974467} step=13856
2022-04-22 13:55.13 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220422132122/model_13856.pt


...

Epoch 39/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-23 03:30.19 [info     ] CQL_20220422132122: epoch=39 step=270192 epoch=39 metrics={'time_sample_batch': 0.0003498303284545969, 'time_algorithm_update': 0.17131448201577051, 'temp_loss': -0.00090944622302515, 'temp': 0.40626227861704783, 'alpha_loss': -0.01116106695183733, 'alpha': 0.026603096482283747, 'critic_loss': 73.52520556871804, 'actor_loss': 12.795844356918721, 'time_step': 0.17202154803358655, 'td_error': 34.377590636009884, 'init_value': -14.611069679260254, 'ave_value': -10.175477616043135} step=270192
2022-04-23 03:30.19 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220422132122/model_270192.pt


Epoch 40/40:   0%|          | 0/6928 [00:00<?, ?it/s]

2022-04-23 03:50.18 [info     ] CQL_20220422132122: epoch=40 step=277120 epoch=40 metrics={'time_sample_batch': 0.00034861521274883806, 'time_algorithm_update': 0.170657395180874, 'temp_loss': -0.0025526430015996643, 'temp': 0.4151186579214646, 'alpha_loss': -0.0061833723156299045, 'alpha': 0.029968462228951394, 'critic_loss': 71.4885447895107, 'actor_loss': 13.600749347315924, 'time_step': 0.1713605998363561, 'td_error': 36.17310510697233, 'init_value': -15.425052642822266, 'ave_value': -11.135538481891} step=277120
2022-04-23 03:50.18 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220422132122/model_277120.pt


[(1,
  {'time_sample_batch': 0.0003420805201519444,
   'time_algorithm_update': 0.10443223916493168,
   'temp_loss': 1.2439730751121016,
   'temp': 0.8144192236624056,
   'alpha_loss': 8.933157939230755,
   'alpha': 0.8295113597125434,
   'critic_loss': 80.33013826481327,
   'actor_loss': 50.652369707479465,
   'time_step': 0.10508638499928402,
   'td_error': 168.37741325843473,
   'init_value': -131.1938018798828,
   'ave_value': -119.46909325804828}),
 (2,
  {'time_sample_batch': 0.0003476064764049257,
   'time_algorithm_update': 0.18429884376481828,
   'temp_loss': -0.018478584296722556,
   'temp': 0.7727976827247727,
   'alpha_loss': 5.844538008385525,
   'alpha': 0.44349783624137384,
   'critic_loss': 458.05218048541707,
   'actor_loss': 169.2142700884689,
   'time_step': 0.18498408794403076,
   'td_error': 301.03213845142466,
   'init_value': -218.14340209960938,
   'ave_value': -199.85732195974467}),
 (3,
  {'time_sample_batch': 0.00034553693568734045,
   'time_algorithm_update'

In [10]:
model.save_model('cqlDet2000_Ep40model_CPUonly.pt')
model.save_policy('cqlDet2000_Ep40_CPUonly.pt')

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [11]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i+2000 for i in range(100)]) #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [12]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset = get_dataset([i*2 for i in range(100)], path="collected_data/rl_stochastic.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=100, n_steps_per_epoch=10000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [13]:
# from d3rlpy.torch_utility import to_cpu
# to_cpu(model)
# model.save_policy("cqlStochpid2000Ep40CPU.pt")
# model.save_model("cqlStochpid2000Ep40modelCPU.pt")

In [14]:
# for key in dir(model):
#     module = getattr(model, key)
#     if isinstance(module, (torch.nn.Module, torch.nn.Parameter)):
#         print(yes)
#         print(key)
# dir(model)
# type(model)
# model.cpu()
# from d3rlpy.algos.torch.base import TorchImplBase
# new_model = TorchImplBase()
# from d3rlpy.torch_utility import _get_attributes
# model._device = "cpu:0"
# print(model._device)


# def my_get_state_dict(impl: Any) -> Dict[str, Any]:
#     rets = {}
#     for key in _get_attributes(impl):
#         obj = getattr(impl, key)
#         if isinstance(obj, (torch.nn.Module, torch.optim.Optimizer)):
#             if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#                 obj.cpu()
#             rets[key] = obj.state_dict()
#     return rets

# torch.save(my_get_state_dict(model), "my_test_model.pt")

# for key in dir(model):
#     obj = getattr(model, key)
#     if isinstance(obj, (torch.nn.Module, torch.nn.Parameter)):
#         obj.cpu()
#         print("convert to cpu")
# model.save_policy("cqlStochpid2000Ep40modelCPU.pt")

# import trace
# tracer = trace.Trace()
# tracer.run('model.save_policy("cqlStochpid2000Ep40modelCPU.pt")')